### [项目在线代码](https://github.com/ksm26/AI-Agentic-Design-Patterns-with-AutoGen/blob/main/L1_Multi-Agent_Conversation_and_Stand-up_Comedy.ipynb)
### [视频地址](https://www.bilibili.com/video/BV1WJ4m137dk/?spm_id_from=333.788.videopod.episodes&vd_source=1d3a7b81d826789081d8b6870d4fff8e&p=2)
### [文档](https://microsoft.github.io/autogen/0.2/docs/Getting-Started)

![attachment:image.png](https://github.com/ksm26/AI-Agentic-Design-Patterns-with-AutoGen/blob/main/images/l1.png?raw=true)

In [24]:
import sys
sys.path.append('/opt/project/KG_Assist_LLM')

### 调用ollama

In [25]:
config_list = [
  {
    "model": "qwen2.5:72b-instruct-q4_0",
    "base_url": "http://192.168.28.5:11434/v1",
    "api_key": "ollama",
    "price": [0, 0], 
  }
]


### 调用openai本地

http://192.168.28.5:8178/v1/

In [ ]:
config_list = [
    {
        "model": "qwen",
        "api_key": "sk-Z9IT6XTbuhNqeWeQwtFj5vlM1NRXsvQ59XObmYU4UIzosvD5",
        "api_type": "openai",
        "base_url": "http://192.168.28.5:8178/v1",
        "n": 1,
        "max_tokens": 4096,
        "temperature": 0,
        "top_p": 0.9,
        "price": [0, 0], 
        "default_headers": {
            "Content-types": "application/json",
            "Authorization": "Bearer xxxx"
        }
    }
]

调用 openai-hk

In [ ]:
# config_list = [
#     {
#         "model": "gpt-3.5-turbo",
#         "api_key": "hk-xxx", # your api key
#         "api_type": "openai",
#         "base_url": "https://api.openai-hk.com/v1",
#         "api_type": "openai",
#     }
# ]

In [28]:
llm_config={"config_list": config_list}

In [29]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER", # always 
)

我们让 ai 回答一个笑话

In [30]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure, here's a joke for you: Why couldn't the bicycle stand up by itself? Because it was two tired!


然后让 ai 重复这个笑话，单个聊天场景下，ai 将不清楚之前问过的笑话，会询问这个笑话是啥，而不是重复之前的笑话

In [31]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke please.", "role": "user"}]
)
print(reply)

Of course! Here is the joke again:

Why did the scarecrow win an award?
Because he was outstanding in his field!


### Conversation
- Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.
- 我们创建两个对话 agent，这两个 agent能够相互对话并且具有记忆能力。

In [32]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.", # 接着上一个笑话的笑点往下说笑话
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [33]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Are you ready for some comedy gold? Let's do this!

--------------------------------------------------------------------------------
joe (to cathy):

Sounds good, Cathy! Let's jump right in. So, why did the scarecrow win an award?
Because he was outstanding in his field!

--------------------------------------------------------------------------------
cathy (to joe):

Cathy, that's a good one! Your turn.

--------------------------------------------------------------------------------


### Print some results
You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

输出聊天历史记录

In [34]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'name': 'joe',
  'role': 'assistant'},
 {'content': "Hey Joe! Are you ready for some comedy gold? Let's do this!",
  'name': 'cathy',
  'role': 'user'},
 {'content': "Sounds good, Cathy! Let's jump right in. So, why did the "
             'scarecrow win an award?\n'
             'Because he was outstanding in his field!',
  'name': 'joe',
  'role': 'assistant'},
 {'content': "Cathy, that's a good one! Your turn.",
  'name': 'cathy',
  'role': 'user'}]


资源消耗统计

In [35]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'total_cost': 0},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 58,
                                                             'cost': 0.00019049999999999997,
                                                             'prompt_tokens': 207,
                                                             'total_tokens': 265},
                                      'total_cost': 0.00019049999999999997}}


对话摘要生成（通常可能是最后一句话）

In [36]:
pprint.pprint(chat_result.summary)

"Cathy, that's a good one! Your turn."


### Get a better summary of the conversation

summary_method和summary_prompt参数会在最后调用，作为 promot 对前面的对话内容进行更的总结

In [37]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Are you ready for some comedy gold? Let's do this!

--------------------------------------------------------------------------------
joe (to cathy):

Sounds good, Cathy! Let's jump right in. So, why did the scarecrow win an award?
Because he was outstanding in his field!

--------------------------------------------------------------------------------
cathy (to joe):

Cathy, that's a good one! Your turn.

--------------------------------------------------------------------------------


In [38]:
pprint.pprint(chat_result.summary)

('Joe and Cathy are cracking jokes and having fun with each other. Joe started '
 "with a scarecrow joke, and now it's Cathy's turn to share a joke.")


### Chat Termination
- Chat can be terminated using a termination conditions.
- 如果我们不知道多少轮结束后会结束， 可以设置一些人为条件来提前控制

In [39]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    # 这里给大模型一个提示， 就是当你说I gotta go的时候，就直接结束会话。
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    # 这里是结束对话的条件函数，当会话中包含I gotta go，就会结束会话。
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)